In [ ]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import cv2
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 5
model = fasterrcnn_resnet50_fpn(pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load('/kaggle/input/fasterrcnn/pytorch/default/1/fasterrcnn_final.pth', map_location=device))
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [2]:
!git clone https://github.com/ifzhang/ByteTrack.git

fatal: destination path 'ByteTrack' already exists and is not an empty directory.


In [3]:
%cd ByteTrack
!pip install -r requirements.txt
!python setup.py develop

/kaggle/working/ByteTrack
  Using cached motmetrics-1.4.0-py3-none-any.whl.metadata (20 kB)
  Using cached filterpy-1.4.5.zip (177 kB)
  Preparing metadata (setup.py) ... done
  Using cached onnx-1.8.1.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement onnxruntime==1.8.0 (from versions: 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.17.0, 1.17.1, 1.17.3, 1.18.0, 1.18.1, 1.19.0, 1.19.2, 1.20.0, 1.20.1, 1.21.0, 1.21.1, 1.22.0)
ERROR: No matching distribution found for onnxruntime==1.8.0
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
   

In [4]:
%cd ..
!pip install opencv-python

/kaggle/working


In [5]:
import sys
sys.path.append('/kaggle/working/ByteTrack')

In [6]:
import numpy as np
np.float = float

In [7]:
!pip install loguru cython_bbox lap thop

In [8]:
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracker import matching
class Args:
    track_thresh = 0.5
    track_buffer = 30
    match_thresh = 0.8
    min_box_area = 10
    mot20 = False
args = Args()

tracker = BYTETracker(args)

In [ ]:
import cv2
import torch
import numpy as np
import os
from yolox.tracker.byte_tracker import BYTETracker
from yolox.tracker import matching

class Args:
    track_thresh = 0.5
    track_buffer = 30
    match_thresh = 0.8
    min_box_area = 10
    mot20 = False
args = Args()

tracker = BYTETracker(args)

video_path = '/kaggle/input/traffic/traffic.mp4'
cap = cv2.VideoCapture(video_path)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracked0.mp4', fourcc, fps, (width, height))

CONF_THRESH = 0.5
CLASS_NAMES = ["others", "car", "van", "bus"]
follow_classes = [1, 2, 3]

frame_id = 0
object_counter_initial = 0
counted_initial_detections = set()
initial_frame_count = 5 


for _ in range(initial_frame_count):
    ret, initial_frame = cap.read()
    if not ret:
        break
    img_initial = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2RGB)
    img_tensor_initial = torch.from_numpy(img_initial / 255.).permute(2, 0, 1).float().unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        outputs_initial = model(img_tensor_initial)[0]
    boxes_initial = outputs_initial['boxes'].cpu().numpy()
    labels_initial = outputs_initial['labels'].cpu().numpy()
    vehicle_indices_initial = np.where(np.isin(labels_initial, follow_classes))[0]
    for idx in vehicle_indices_initial:
        box = tuple(map(int, boxes_initial[idx]))
        if box not in counted_initial_detections:
            counted_initial_detections.add(box)
            object_counter_initial += 1
    frame_id += 1 

object_counter = object_counter_initial
counted_ids = set()

result_path = 'tracking_results.txt'
if os.path.exists(result_path):
    os.remove(result_path)
result_file = open(result_path, 'w')

if hasattr(tracker, 'reset'):
    tracker.reset()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_tensor = torch.from_numpy(img / 255.).permute(2, 0, 1).float().unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        outputs = model(img_tensor)[0]

    boxes = outputs['boxes'].cpu().numpy()
    scores = outputs['scores'].cpu().numpy()
    labels = outputs['labels'].cpu().numpy()

    vehicle_indices = np.where(np.isin(labels, follow_classes))[0]
    dets = []
    det_classes = []
    for idx in vehicle_indices:
        x1, y1, x2, y2 = boxes[idx]
        score = scores[idx]
        dets.append([x1, y1, x2, y2, score])
        det_classes.append(int(labels[idx]))

    dets = np.array(dets)
    if len(dets) == 0:
        dets = np.empty((0, 5))

    img_info = [frame.shape[0], frame.shape[1]]
    img_size = (frame.shape[0], frame.shape[1])

    online_targets = tracker.update(dets, img_info, img_size)

    active_ids = []
    for t in online_targets:
        tlwh = t.tlwh
        track_id = t.track_id
        active_ids.append(track_id)

        is_new = False
        if track_id not in counted_ids:
            counted_ids.add(track_id)
            object_counter += 1
            is_new = True

        t_box = np.array([tlwh[0], tlwh[1], tlwh[0] + tlwh[2], tlwh[1] + tlwh[3]])
        best_iou = 0
        best_cls = -1
        for det_box, cls in zip(dets, det_classes):
            xx1 = max(t_box[0], det_box[0])
            yy1 = max(t_box[1], det_box[1])
            xx2 = min(t_box[2], det_box[2])
            yy2 = min(t_box[3], det_box[3])
            w = max(0., xx2 - xx1)
            h = max(0., yy2 - yy1)
            intersection = w * h
            area1 = (t_box[2] - t_box[0]) * (t_box[3] - t_box[1])
            area2 = (det_box[2] - det_box[0]) * (det_box[3] - det_box[1])
            union = area1 + area2 - intersection
            iou = intersection / union if union > 0 else 0
            if iou > best_iou:
                best_iou = iou
                best_cls = cls

        class_name = CLASS_NAMES[best_cls] if 0 <= best_cls < len(CLASS_NAMES) else "unknown"
        x1, y1, w, h = [int(i) for i in tlwh]

        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        label = f"{track_id}: {class_name}"
        cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

        result_file.write(f"{frame_id},{class_name},{x1},{y1},{w},{h}\n")

    active_vehicle_count = len(active_ids)
    total_unique_count = len(counted_ids) + object_counter_initial

    density_level = ""
    if active_vehicle_count <= 5:
        density_level = "Low"
    elif active_vehicle_count <= 10:
        density_level = "Medium"
    else:
        density_level = "High"

    density_text = f"Traffic: {density_level}"
    text_size_density = cv2.getTextSize(density_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
    text_x_density = (width - text_size_density[0]) // 2
    text_y_density = height - 5
    cv2.putText(frame, density_text, (text_x_density, text_y_density), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)

    out.write(frame)

cap.release()
out.release()
result_file.close()